<a href="https://colab.research.google.com/github/vichidamba/maksimus_emotion-recognition/blob/main/MMER_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi Modal Emotion Recognition - Video Only
This notebook implements only the video module from the paper 
https://www.mdpi.com/2076-3417/12/1/327
Also, Only sequential module is implemented since it has been reported for higher accuracy.

######Note: This notebook requires GPU and more RAM, So Iutilized google colab's pro+ resources to run this.

Import google drive for importing ravdess dataset

In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


Download Openface2.0


The following code is a little modified to run with CUDA 10. Also install openface in a seperate directory altogether.
The following block has to take 20-30+ minutes to run. If it ran fatser than that, it probably hasn't installed something properly.

In [2]:
import os
from os.path import exists, join, basename, splitext

################# Need to revert back to CUDA 10.0 ##################
# Thanks to http://aconcaguasci.blogspot.com/2019/12/setting-up-cuda-100-for-mxnet-on-google.html
#Uninstall the current CUDA version
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

#Download CUDA 10.0
!wget  --no-clobber https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
#install CUDA kit dpkg
!dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!apt-get update
!apt-get install cuda-10-0
#Slove libcurand.so.10 error
!wget --no-clobber http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
#-nc, --no-clobber: skip downloads that would download to existing files.
!apt install ./nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
!apt-get update


git_repo_url = 'https://github.com/TadasBaltrusaitis/OpenFace.git'
project_name = splitext(basename(git_repo_url))[0]
# clone openface
!git clone -q --depth 1 $git_repo_url

# install new CMake becaue of CUDA10
!wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
!tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

# Get newest GCC
!sudo apt-get update
!sudo apt-get install build-essential 
!sudo apt-get install g++-8

!sudo apt-key del 7fa2af80
!sudo apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/3bf863cc.pub

# install python dependencies
!pip install -q youtube-dl

# Finally, actually install OpenFace
!cd OpenFace && bash ./download_models.sh && sudo bash ./install.sh

Streaming output truncated to the last 5000 lines.
  inflating: opencv-4.1.0/samples/cpp/tutorial_code/calib3d/real_time_pose_estimation/src/ModelRegistration.h  
  inflating: opencv-4.1.0/samples/cpp/tutorial_code/calib3d/real_time_pose_estimation/src/PnPProblem.cpp  
  inflating: opencv-4.1.0/samples/cpp/tutorial_code/calib3d/real_time_pose_estimation/src/PnPProblem.h  
  inflating: opencv-4.1.0/samples/cpp/tutorial_code/calib3d/real_time_pose_estimation/src/RobustMatcher.cpp  
  inflating: opencv-4.1.0/samples/cpp/tutorial_code/calib3d/real_time_pose_estimation/src/RobustMatcher.h  
  inflating: opencv-4.1.0/samples/cpp/tutorial_code/calib3d/real_time_pose_estimation/src/Utils.cpp  
  inflating: opencv-4.1.0/samples/cpp/tutorial_code/calib3d/real_time_pose_estimation/src/Utils.h  
  inflating: opencv-4.1.0/samples/cpp/tutorial_code/calib3d/real_time_pose_estimation/src/main_detection.cpp  
  inflating: opencv-4.1.0/samples/cpp/tutorial_code/calib3d/real_time_pose_estimation/src/main

In [3]:
ravdess_zip="/content/drive/MyDrive/videos.zip"

In [4]:
!unzip "/content/drive/MyDrive/videos.zip" -d "/content/tempo"

Archive:  /content/drive/MyDrive/videos.zip
   creating: /content/tempo/videos/
  inflating: /content/tempo/videos/01-01-07-02-02-02-03.mp4  
  inflating: /content/tempo/__MACOSX/videos/._01-01-07-02-02-02-03.mp4  
  inflating: /content/tempo/videos/01-01-04-02-02-01-17.mp4  
  inflating: /content/tempo/__MACOSX/videos/._01-01-04-02-02-01-17.mp4  
  inflating: /content/tempo/videos/01-01-07-02-02-02-17.mp4  
  inflating: /content/tempo/__MACOSX/videos/._01-01-07-02-02-02-17.mp4  
  inflating: /content/tempo/videos/01-01-04-02-02-01-03.mp4  
  inflating: /content/tempo/__MACOSX/videos/._01-01-04-02-02-01-03.mp4  
  inflating: /content/tempo/videos/01-01-03-02-02-01-13.mp4  
  inflating: /content/tempo/__MACOSX/videos/._01-01-03-02-02-01-13.mp4  
  inflating: /content/tempo/videos/01-01-03-02-02-01-07.mp4  
  inflating: /content/tempo/__MACOSX/videos/._01-01-03-02-02-01-07.mp4  
  inflating: /content/tempo/videos/01-01-03-01-01-01-07.mp4  
  inflating: /content/tempo/__MACOSX/videos/._01

In [5]:
! pip install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=7034b96d631eb3912bf5297acd0b495fa3b47e5ac5d4c093e5b86e68eab7f44d
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg


In [6]:
!pip install ekphrasis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 2.0 MB/s 
     |████████████████████████████████| 53 kB 14 kB/s 
     |████████████████████████████████| 45 kB 3.4 MB/s 


In [7]:
! pip install git+https://github.com/huggingface/datasets.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-bmemsler
  Running command git clone -q https://github.com/huggingface/datasets.git /tmp/pip-req-build-bmemsler
     |████████████████████████████████| 212 kB 5.0 MB/s 
     |████████████████████████████████| 140 kB 68.1 MB/s 
     |████████████████████████████████| 1.1 MB 60.7 MB/s 
     |████████████████████████████████| 101 kB 13.9 MB/s 
     |████████████████████████████████| 596 kB 69.1 MB/s 
     |████████████████████████████████| 127 kB 103.1 MB/s 
     |████████████████████████████████| 144 kB 92.6 MB/s 
     |████████████████████████████████| 271 kB 95.8 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
  Created wheel for datasets: filename=datasets-2.3.3.dev0-py3-none-any.whl size=365268 sha256=51ed0e90bd1ba399204d890698141d540a50808fdd8f57df87a5e315a935bf60
  Stored in directory: /tmp/pip-e

In [8]:
! pip install git+https://github.com/huggingface/transformers.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-pcaup028
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-pcaup028
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 5.1 MB/s 
  Created wheel for transformers: filename=transformers-4.21.0.dev0-py3-none-any.whl size=4610948 sha256=d6bbcd8a08aa7e6b363b2296353ef9fe4a6aed350827b4354af3e0efacdd086b
  Stored in directory: /tmp/pip-ephem-wheel-cache-o08q0tmj/wheels/90/a5/44/6bcd83827c8a60628c5ad602f429cd5076bcce5f2a90054947
Successfully built transformers


Clone MMER Repository in a seperate directory

In [9]:
! git clone  https://github.com/cristinalunaj/MMEmotionRecognition

Cloning into 'MMEmotionRecognition'...
remote: Enumerating objects: 339, done.
remote: Counting objects: 100% (334/334), done.
remote: Compressing objects: 100% (235/235), done.
remote: Total 339 (delta 91), reused 313 (delta 73), pack-reused 5
Receiving objects: 100% (339/339), 24.57 MiB | 31.40 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [10]:
! pip install -r "/content/MMEmotionRecognition/requirements.txt"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git (to revision 857ab55c01cf7213bc1822933cd2ef2b7552bac4) to /tmp/pip-install-ihhfl6b3/transformers_be99fe3a087e498a8f611375a87cedc9
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-install-ihhfl6b3/transformers_be99fe3a087e498a8f611375a87cedc9
  Running command git rev-parse -q --verify 'sha^857ab55c01cf7213bc1822933cd2ef2b7552bac4'
  Running command git fetch -q https://github.com/huggingface/transformers.git 857ab55c01cf7213bc1822933cd2ef2b7552bac4
  Running command git checkout -q 857ab55c01cf7213bc1822933cd2ef2b7552bac4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 126 kB 5.1 MB/s 
     |████████████████████████████████| 149 kB 45.3 MB/s 
     |█████████████████████████████

Extract Action units for all the ravdess videos and save the results under ravdess dataset directory


This should run for about an hour or longer.

In [11]:
!python3 MMEmotionRecognition/src/Video/OpenFace/AUsFeatureExtractor.py --videos_dir /content/Ravdess_dir/videos --openFace_path /content/OpenFace --out_dir /content/Ravdess_dir/Extracted_AUs --out_dir_processed /content/Ravdess_dir/processed_AUs

Streaming output truncated to the last 5000 lines.
Closing input reader
Closed successfully
Postprocessing the Action Unit predictions
Could not find the HAAR face detector location
Reading the landmark detector/tracker from: /content/OpenFace/build/bin/model/main_ceclm_general.txt
Reading the landmark detector module from: /content/OpenFace/build/bin/model/cen_general.txt
Reading the PDM module from: /content/OpenFace/build/bin/model/pdms/In-the-wild_aligned_PDM_68.txt....Done
Reading the Triangulations module from: /content/OpenFace/build/bin/model/tris_68.txt....Done
Reading the intensity CEN patch experts from: /content/OpenFace/build/bin/model/patch_experts/cen_patches_0.25_of.dat....Done
Reading the intensity CEN patch experts from: /content/OpenFace/build/bin/model/patch_experts/cen_patches_0.35_of.dat....Done
Reading the intensity CEN patch experts from: /content/OpenFace/build/bin/model/patch_experts/cen_patches_0.50_of.dat....Done
Reading the intensity CEN patch experts from:

Prepare the dataset and keep a check on your directories and file paths, Since the output is going to be a csv or txt file full of video paths associated with its labels. 

In [14]:
! python3 /content/MMEmotionRecognition/src/Video/models/sequenceLearning/frontend/RAVDESS_AUs/frontend_ravdess_5CV.py --AUs_dir /content/Ravdess_dir/processed_AUs --out_dir MMEmotionRecognition/src/Video/models/sequenceLearning/datasets/RAVDESS_AUs

Train the model, Change the hyperparamters from the Ravdess_au.json file.
This takes approx 45mins to 90 mins.

In [15]:
! python3 /content/MMEmotionRecognition/src/Video/models/sequenceLearning/workflow/run.py train /content/MMEmotionRecognition/src/Video/models/sequenceLearning/conf/RAVDESS_AUs.json --kfolds 5

Running on device CUDA
SCRIPT: run.py
Options...
  mode: train
  conf: /content/MMEmotionRecognition/src/Video/models/sequenceLearning/conf/RAVDESS_AUs.json
  pretrained: None
  finetune: False
  kfolds: 5
------------------------------
Config-file params...
  name: RAVDESS_AUs
  data_type: pathCSV
  model_name: Sequence1Modal
  inference_data: val
  input_size: 35
  model_params: {'encoder': {'dim': 50, 'layers': 2, 'dropout': 0.3, 'bidirectional': True}, 'attention': {'layers': 2, 'dropout': 0.3, 'activation': 'tanh', 'context': False}}
  preprocessor: None
  batch_size: 64
  lr: 0.001
  weight_decay: 0.0
  patience: 30
  min_change: 0.0
  epochs: 300
  base: 0.0
  clip_norm: 1
  seed: 2020
  disable_cache: True
------------------------------
Process initiated at time:  18:52:51
 INICIALIZANDO SEMILLA RANDOM!!! 
******************************
[FOLD 1 OUT OF 5]
******************************
label_map:  {'angry': 0, 'calm': 1, 'disgust': 2, 'fear': 3, 'happy': 4, 'neutral': 5, 'sad': 

Run Inference, Edit parameters on posterior json file.

In [19]:
! python3 /content/MMEmotionRecognition/src/Video/models/sequenceLearning/workflow/run.py inference /content/MMEmotionRecognition/src/Video/models/sequenceLearning/conf/RAVDESS_AUs_posteriors.json --kfolds 5 --pretrained /content/MMEmotionRecognition/src/Video/models/sequenceLearning/out/trained

Running on device CUDA
SCRIPT: run.py
Options...
  mode: inference
  conf: /content/MMEmotionRecognition/src/Video/models/sequenceLearning/conf/RAVDESS_AUs_posteriors.json
  pretrained: /content/MMEmotionRecognition/src/Video/models/sequenceLearning/out/trained
  finetune: False
  kfolds: 5
------------------------------
Config-file params...
  name: RAVDESS_AUs
  data_type: pathCSV
  model_name: Sequence1Modal
  inference_data: val
  OUT_PATH: /content/MMEmotionRecognition/src/Video/models/sequenceLearning
  TRAINED_PATH: MMEmotionRecognition/src/Video/models/sequenceLearning/out/trained
  input_size: 35
  model_params: {'encoder': {'dim': 50, 'layers': 2, 'dropout': 0.3, 'bidirectional': True}, 'attention': {'layers': 2, 'dropout': 0.3, 'activation': 'tanh', 'context': False}}
  preprocessor: None
  batch_size: 64
  lr: 0.001
  weight_decay: 0.0
  patience: 30
  min_change: 0.0
  epochs: 300
  base: 0.0
  clip_norm: 1
  seed: 2020
  disable_cache: True
------------------------------


Prediction results are out